In [ ]:
"""
# Parkinson's Disease Prediction using Advanced Lightweight Ensemble
# Combining the reliable training approach with MobileNetV2 + ShuffleNetV2 + GhostNet

This notebook implements an optimized lightweight ensemble model to predict Parkinson's Disease
from fMRI images, designed for small, imbalanced datasets with reliable training.
"""

# Mount Google Drive (for Colab)
try:
    from google.colab import drive
    import os
    if not os.path.exists('/content/drive'):
        print("Mounting Google Drive...")
        drive.mount('/content/drive')
    else:
        print("Google Drive already mounted.")
except ImportError:
    pass  # Not running in Colab, skip mounting

# All imports and code below (no pip install/uninstall, one-cell friendly)
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score, roc_curve, auc,
    roc_auc_score, precision_recall_curve, average_precision_score
)
import glob
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.applications import MobileNetV2

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# --- GOOGLE DRIVE DATASET PATH SETUP ---
PD_DIR = "/content/drive/MyDrive/kaggle dataset/Disease_Prediction/PD_new"
CONTROL_DIR = "/content/drive/MyDrive/kaggle dataset/Disease_Prediction/HC_new"

# --- VERIFY DATASET CONTENTS ---
print("PD images:", len(os.listdir(PD_DIR)) if os.path.exists(PD_DIR) else 0)
print("Control images:", len(os.listdir(CONTROL_DIR)) if os.path.exists(CONTROL_DIR) else 0)

# Constants
IMG_SIZE = 224  # Standard size for pre-trained models
BATCH_SIZE = 16
AUTOTUNE = tf.data.AUTOTUNE

def get_image_files(directory):
    if not os.path.exists(directory):
        print(f"Warning: Directory {directory} does not exist")
        return []
    jpg_files = glob.glob(os.path.join(directory, "*.jpg"))
    png_files = glob.glob(os.path.join(directory, "*.png"))
    return jpg_files + png_files

pd_images = get_image_files(PD_DIR)
control_images = get_image_files(CONTROL_DIR)

print(f"Parkinson's images: {len(pd_images)}")
print(f"Control images: {len(control_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    if tf.strings.regex_full_match(file_path, ".*\.jpg"):
        img = tf.image.decode_jpeg(img, channels=3)
    else:
        img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.ensure_shape(img, [IMG_SIZE, IMG_SIZE, 3])
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.per_image_standardization(img)
    img = (img - tf.reduce_min(img)) / (tf.reduce_max(img) - tf.reduce_min(img))
    return img, label

# Test-time augmentation for more robust predictions
def tta_predict(model, images, num_augmentations=10):
    """Test-time augmentation for more reliable predictions"""
    # Create a basic augmentation pipeline for TTA
    tta_aug = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.1),
        layers.RandomContrast(0.2)
    ])

    # Initialize predictions array
    predictions = model.predict(images)  # Original prediction

    # Add predictions with augmentations
    for _ in range(num_augmentations-1):
        aug_images = tta_aug(images, training=True)
        predictions += model.predict(aug_images)

    # Average predictions
    return predictions / num_augmentations

# Function to find the optimal threshold for classification
def find_optimal_threshold(y_true, y_pred_prob):
    """Find the optimal threshold for binary classification"""
    thresholds = np.arange(0.1, 0.9, 0.05)
    best_f1 = 0
    best_threshold = 0.5

    for threshold in thresholds:
        y_pred = (y_pred_prob > threshold).astype(int)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    print(f"Optimal threshold found: {best_threshold:.2f} with F1 score: {best_f1:.4f}")
    return best_threshold

# Helper function to create a dense block with regularization
def _add_dense_block(x, units, dropout_rate=0.3):
    """Add a dense layer with batch normalization and dropout"""
    x = layers.Dense(
        units,
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.001)
    )(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.BatchNormalization()(x)
    return x

# ShuffleNet block implementation
def create_shufflenet_block(x, out_channels, stride=1):
    """Create a ShuffleNet block"""
    input_channels = x.shape[-1]

    # Create shortcut connection
    if stride == 1 and input_channels == out_channels:
        shortcut = x
    else:
        shortcut = layers.AveragePooling2D(pool_size=3, strides=stride, padding='same')(x)
        shortcut = layers.Conv2D(out_channels, 1, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    # Channel shuffle
    x = layers.Conv2D(out_channels, 1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Depthwise convolution
    x = layers.DepthwiseConv2D(3, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Pointwise convolution
    x = layers.Conv2D(out_channels, 1, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Add shortcut
    x = layers.Add()([shortcut, x])
    x = layers.ReLU()(x)
    return x

# Ghost module implementation
def create_ghost_module(x, out_channels, kernel_size=3, stride=1):
    """Create a Ghost module with consistent output shapes"""
    # Ensure out_channels is even for proper splitting
    primary_channels = out_channels // 2
    ghost_channels = out_channels - primary_channels

    # Primary convolution
    primary = layers.Conv2D(primary_channels, kernel_size, strides=stride, padding='same')(x)
    primary = layers.BatchNormalization()(primary)
    primary = layers.ReLU()(primary)

    # Ghost convolution - generate additional features from primary
    ghost = layers.DepthwiseConv2D(kernel_size, strides=1, padding='same')(primary)
    ghost = layers.BatchNormalization()(ghost)
    ghost = layers.ReLU()(ghost)

    # If we need more ghost channels, pad with zeros or duplicate
    if ghost_channels > primary_channels:
        # Add extra channels by duplicating some ghost features
        extra_channels = ghost_channels - primary_channels
        ghost_extra = layers.Conv2D(extra_channels, 1, padding='same')(ghost)
        ghost = layers.Concatenate()([ghost, ghost_extra])
    elif ghost_channels < primary_channels:
        # Reduce ghost channels
        ghost = layers.Conv2D(ghost_channels, 1, padding='same')(ghost)

    # Concatenate primary and ghost features
    return layers.Concatenate()([primary, ghost])

# Create the advanced lightweight ensemble model
def create_mobile_shuffle_ghost_ensemble():
    """Creates an ultra-high performance ensemble with attention mechanisms"""
    inputs = layers.Input(shape=(224, 224, 3))  # Explicitly use 224x224

    # MobileNetV2 branch with fine-tuning
    mobilenet = tf.keras.applications.MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3),  # Explicitly use 224x224 for MobileNetV2
        name='mobilenet_branch'
    )
    # Fine-tune last 30 layers for better performance
    for layer in mobilenet.layers[:-30]:
        layer.trainable = False

    mobilenet_output = mobilenet(inputs)
    mobilenet_output = layers.GlobalAveragePooling2D()(mobilenet_output)
    mobilenet_output = layers.BatchNormalization()(mobilenet_output)
    mobilenet_output = layers.Dropout(0.3)(mobilenet_output)

    # Enhanced ShuffleNetV2 branch
    x = layers.Conv2D(32, 3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # More ShuffleNet blocks for better feature extraction
    x = create_shufflenet_block(x, 64, stride=2)   # 32 -> 64
    x = create_shufflenet_block(x, 128, stride=2)  # 64 -> 128
    x = create_shufflenet_block(x, 256, stride=2)  # 128 -> 256
    x = create_shufflenet_block(x, 256, stride=1)  # 256 -> 256 (same channels)

    shufflenet_output = layers.GlobalAveragePooling2D()(x)
    shufflenet_output = layers.BatchNormalization()(shufflenet_output)
    shufflenet_output = layers.Dropout(0.3)(shufflenet_output)

    # Enhanced GhostNet branch
    x = layers.Conv2D(24, 3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # More Ghost modules for better feature extraction
    x = create_ghost_module(x, 48, stride=2)   # 24 -> 48
    x = create_ghost_module(x, 96, stride=2)   # 48 -> 96
    x = create_ghost_module(x, 192, stride=2)  # 96 -> 192
    x = create_ghost_module(x, 192, stride=1)  # 192 -> 192 (same channels)

    ghostnet_output = layers.GlobalAveragePooling2D()(x)
    ghostnet_output = layers.BatchNormalization()(ghostnet_output)
    ghostnet_output = layers.Dropout(0.3)(ghostnet_output)

    # Advanced feature fusion with attention mechanism
    combined_features = layers.Concatenate()([mobilenet_output, shufflenet_output, ghostnet_output])

    # Self-attention mechanism for feature weighting
    attention_weights = layers.Dense(combined_features.shape[-1], activation='sigmoid', name='attention_weights')(combined_features)
    weighted_features = layers.Multiply(name='weighted_features')([combined_features, attention_weights])

    # Residual connection
    final_features = layers.Add(name='residual_connection')([combined_features, weighted_features])

    # Ultra-high performance classification head
    x = layers.BatchNormalization()(final_features)
    x = _add_dense_block(x, 512, dropout_rate=0.5)
    x = _add_dense_block(x, 256, dropout_rate=0.4)
    x = _add_dense_block(x, 128, dropout_rate=0.3)
    x = _add_dense_block(x, 64, dropout_rate=0.2)

    outputs = layers.Dense(1, activation='sigmoid', name='final_output')(x)

    return Model(inputs=inputs, outputs=outputs, name='ultra_performance_ensemble')

# Cyclic learning rate function
def cyclic_lr_schedule(epoch, initial_lr=5e-5, max_lr=1e-4, step_size=5):
    """Cyclic learning rate schedule"""
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = initial_lr + (max_lr - initial_lr) * np.maximum(0, (1 - x))
    return float(lr)

# Helper function to save all visualizations
def save_visualizations(test_labels, y_pred, y_pred_prob, history, model_dir):
    """Create and save all visualizations for model evaluation"""
    # 1. Confusion Matrix
    cm = confusion_matrix(test_labels, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Control', "Parkinson's"],
                yticklabels=['Control', "Parkinson's"])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.savefig(os.path.join(model_dir, 'confusion_matrix.png'))
    plt.close()

    # 2. Training History
    plt.figure(figsize=(16, 12))

    # Accuracy plot
    plt.subplot(2, 2, 1)
    plt.plot(history.history['accuracy'], label='Training')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.title('Accuracy', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    # AUC plot
    plt.subplot(2, 2, 2)
    plt.plot(history.history['auc'], label='Training')
    plt.plot(history.history['val_auc'], label='Validation')
    plt.title('Area Under Curve (AUC)', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('AUC', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    # Precision/Recall plot
    plt.subplot(2, 2, 3)
    plt.plot(history.history['precision'], label='Training Precision')
    plt.plot(history.history['val_precision'], label='Validation Precision')
    plt.plot(history.history['recall'], label='Training Recall')
    plt.plot(history.history['val_recall'], label='Validation Recall')
    plt.title('Precision and Recall', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Score', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    # Loss plot
    plt.subplot(2, 2, 4)
    plt.plot(history.history['loss'], label='Training')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title('Loss', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(os.path.join(model_dir, 'training_metrics.png'), dpi=300)
    plt.close()

    # 3. ROC Curve
    fpr, tpr, _ = roc_curve(test_labels, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title('Receiver Operating Characteristic', fontsize=16)
    plt.legend(loc="lower right", fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig(os.path.join(model_dir, 'roc_curve.png'), dpi=300)
    plt.close()

    # 4. Precision-Recall Curve
    precision_values, recall_values, _ = precision_recall_curve(test_labels, y_pred_prob)
    avg_precision = average_precision_score(test_labels, y_pred_prob)

    plt.figure(figsize=(10, 8))
    plt.plot(recall_values, precision_values, lw=2, label=f'Precision-Recall curve (AP = {avg_precision:.3f})')
    plt.xlabel('Recall', fontsize=14)
    plt.ylabel('Precision', fontsize=14)
    plt.title('Precision-Recall Curve', fontsize=16)
    plt.legend(loc="best", fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig(os.path.join(model_dir, 'precision_recall_curve.png'), dpi=300)
    plt.close()

    # 5. Save metrics to CSV
    pd.DataFrame(history.history).to_csv(os.path.join(model_dir, 'training_metrics.csv'), index=False)

# Create a main function with proper flow
def main():
    # Create output directory
    model_dir = '/content/drive/MyDrive/parkinsons_lightweight_model_results'
    os.makedirs(model_dir, exist_ok=True)
    print(f"📁 Saving all results to: {model_dir}")

    print("\n=== PREPARING DATASET ===")
    # Get and balance image datasets
    pd_images = get_image_files(PD_DIR)
    control_images = get_image_files(CONTROL_DIR)

    try:
        print(f"Original dataset - PD: {len(pd_images)}, Control: {len(control_images)}")

        # Create data splits
        all_files = pd_images + control_images
        labels = [1] * len(pd_images) + [0] * len(control_images)

        train_files, temp_files, train_labels, temp_labels = train_test_split(
            all_files, labels, test_size=0.3,
            random_state=42, stratify=labels
        )
        val_files, test_files, val_labels, test_labels = train_test_split(
            temp_files, temp_labels, test_size=0.5,
            random_state=42, stratify=temp_labels
        )

        print(f"Split dataset:")
        pd_train = sum(1 for l in train_labels if l == 1)
        control_train = sum(1 for l in train_labels if l == 0)
        pd_val = sum(1 for l in val_labels if l == 1)
        control_val = sum(1 for l in val_labels if l == 0)
        pd_test = sum(1 for l in test_labels if l == 1)
        control_test = sum(1 for l in test_labels if l == 0)
        print(f"Train set: {pd_train} PD, {control_train} control")
        print(f"Validation set: {pd_val} PD, {control_val} control")
        print(f"Test set: {pd_test} PD, {control_test} control")

        # Create datasets
        train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
        val_ds = tf.data.Dataset.from_tensor_slices((val_files, val_labels))
        test_ds = tf.data.Dataset.from_tensor_slices((test_files, test_labels))

        # Map to images
        train_ds = train_ds.map(lambda x, y: process_path(x, y), num_parallel_calls=AUTOTUNE)
        val_ds = val_ds.map(lambda x, y: process_path(x, y), num_parallel_calls=AUTOTUNE)
        test_ds = test_ds.map(lambda x, y: process_path(x, y), num_parallel_calls=AUTOTUNE)

        # Calculate class weights for the imbalanced dataset
        total_samples = pd_train + control_train
        weight_for_0 = (total_samples / (2.0 * control_train)) * 0.5  # Control class
        weight_for_1 = (total_samples / (2.0 * pd_train)) * 1.5       # PD class
        class_weight = {0: weight_for_0, 1: weight_for_1}
        print(f"Class weights - Control: {weight_for_0:.2f}, PD: {weight_for_1:.2f}")

        # Apply simple augmentation to avoid tensor shape issues
        def safe_augment(image, label):
            # Use simple augmentations that won't cause shape issues
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_flip_up_down(image)
            image = tf.image.random_brightness(image, 0.2)
            image = tf.image.random_contrast(image, 0.8, 1.2)
            image = tf.image.random_saturation(image, 0.8, 1.2)
            image = tf.image.random_hue(image, 0.1)
            return image, label

        # Apply augmentation then batch
        train_ds = train_ds.map(safe_augment, num_parallel_calls=AUTOTUNE)
        train_ds = train_ds.shuffle(buffer_size=1000, seed=42).repeat()
        train_ds = train_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

        # For validation and test, just batch without augmentation
        val_ds = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
        test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

        print("\n=== CREATING ADVANCED ENSEMBLE MODEL ===")
        model = create_mobile_shuffle_ghost_ensemble()
        print(f"Model created with {model.count_params():,} parameters")

        # Compile model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
            loss='binary_crossentropy',
            metrics=[
                'accuracy',
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall'),
                tf.keras.metrics.AUC(name='auc')
            ]
        )

        # Set up callbacks
        callbacks = [
            # Model checkpoints
            ModelCheckpoint(
                filepath=os.path.join(model_dir, 'best_model.h5'),
                monitor='val_auc', mode='max',
                save_best_only=True, verbose=1
            ),
            # Training optimization
            EarlyStopping(
                monitor='val_auc', patience=15,
                restore_best_weights=True, verbose=1
            ),
            LearningRateScheduler(cyclic_lr_schedule, verbose=1),
            ReduceLROnPlateau(
                monitor='val_loss', factor=0.7, patience=7,
                min_lr=1e-6, verbose=1
            )
        ]

        # Calculate steps per epoch to use approximately all data
        steps_per_epoch = len(train_files) // BATCH_SIZE + 1

        # Train model
        print("\nTraining advanced ensemble model...")
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=30,
            callbacks=callbacks,
            steps_per_epoch=steps_per_epoch,
            class_weight=class_weight
        )

        # Evaluate model
        print("\n=== EVALUATING MODEL ===")
        test_results = model.evaluate(test_ds, verbose=1)
        print(f"Test accuracy: {test_results[1]:.4f}")
        print(f"Test precision: {test_results[2]:.4f}")
        print(f"Test recall: {test_results[3]:.4f}")
        print(f"Test AUC: {test_results[4]:.4f}")

        # Test-time augmentation evaluation
        print("\nPerforming test-time augmentation for more robust predictions...")
        test_images = np.vstack([images for images, _ in test_ds])
        test_labels = np.hstack([labels.numpy() for _, labels in test_ds])

        y_pred_prob = tta_predict(model, test_images)

        # Find optimal threshold
        threshold = find_optimal_threshold(test_labels, y_pred_prob)
        y_pred = (y_pred_prob > threshold).astype(int)

        # Ensure predictions match true labels length
        y_pred = y_pred[:len(test_labels)]
        y_pred_prob = y_pred_prob[:len(test_labels)]

        # Calculate and display metrics
        metrics = {
            'accuracy': accuracy_score(test_labels, y_pred),
            'precision': precision_score(test_labels, y_pred, zero_division=0),
            'recall': recall_score(test_labels, y_pred, zero_division=0),
            'f1': f1_score(test_labels, y_pred, zero_division=0),
            'roc_auc': roc_auc_score(test_labels, y_pred_prob) if len(np.unique(test_labels)) > 1 else 0.5
        }

        print("\n🏆 === FINAL RESULTS WITH ADVANCED ENSEMBLE ===")
        for metric_name, value in metrics.items():
            print(f"{metric_name.capitalize()}: {value:.4f}")

        print("\nClassification Report:")
        print(classification_report(test_labels, y_pred,
                                   target_names=['Control', "Parkinson's"],
                                   zero_division=0))

        # Save visualizations and model
        save_visualizations(test_labels, y_pred, y_pred_prob, history, model_dir)

        # Save final model
        model_save_path = os.path.join(model_dir, 'advanced_ensemble_model.keras')
        model.save(model_save_path)
        print(f"\nAdvanced ensemble model saved to {model_save_path}")

        return model, history

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return None, None

# Call the main function when script is run
if __name__ == "__main__":
    main()

Google Drive already mounted.
PD images: 290
Control images: 26
Parkinson's images: 215
Control images: 26
📁 Saving all results to: /content/drive/MyDrive/parkinsons_lightweight_model_results

=== PREPARING DATASET ===
Original dataset - PD: 215, Control: 26
Split dataset:
Train set: 150 PD, 18 control
Validation set: 32 PD, 4 control
Test set: 33 PD, 4 control
Class weights - Control: 2.33, PD: 0.84

=== CREATING ADVANCED ENSEMBLE MODEL ===
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Model created with 6,924,953 parameters

Training advanced ensemble model...

Epoch 1: LearningRateScheduler setting learning rate to 4.999999873689376e-05.
Epoch 1/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4421 - auc: 0.4907 - loss: 2.3791 - precision: 0.8761 - recall: 0.4270
Epoch 1: val_auc improved from -inf to 0.77344, saving model to /content/drive/MyDrive/parkinsons_lightweight_model_results/best_model.h5


11/11 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.4432 - auc: 0.4873 - loss: 2.3795 - precision: 0.8762 - recall: 0.4291 - val_accuracy: 0.8889 - val_auc: 0.7734 - val_loss: 1.7937 - val_precision: 0.8889 - val_recall: 1.0000 - learning_rate: 5.0000e-05

Epoch 2: LearningRateScheduler setting learning rate to 5.9999998989515005e-05.
Epoch 2/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5020 - auc: 0.5932 - loss: 2.2273 - precision: 0.8980 - recall: 0.4976
Epoch 2: val_auc improved from 0.77344 to 0.91797, saving model to /content/drive/MyDrive/parkinsons_lightweight_model_results/best_model.h5


11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.5004 - auc: 0.5888 - loss: 2.2288 - precision: 0.8975 - recall: 0.4959 - val_accuracy: 0.8889 - val_auc: 0.9180 - val_loss: 1.8497 - val_precision: 0.8889 - val_recall: 1.0000 - learning_rate: 6.0000e-05

Epoch 3: LearningRateScheduler setting learning rate to 7.59999990905635e-05.
Epoch 3/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5054 - auc: 0.5480 - loss: 2.2157 - precision: 0.8856 - recall: 0.4997
Epoch 3: val_auc did not improve from 0.91797
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.5059 - auc: 0.5518 - loss: 2.2135 - precision: 0.8875 - recall: 0.5000 - val_accuracy: 0.8889 - val_auc: 0.9180 - val_loss: 1.9185 - val_precision: 0.8889 - val_recall: 1.0000 - learning_rate: 7.6000e-05

Epoch 4: LearningRateScheduler setting learning rate to 9.03999986499548e-05.
Epoch 4/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 942ms/step - accuracy: 0.4955 - auc: 0.5364 - loss: 2.2588 - precision: 0.8708 - recall: 0.5107
Epoch 4: va

11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.5610 - auc: 0.7198 - loss: 2.0691 - precision: 0.9372 - recall: 0.5500 - val_accuracy: 0.9444 - val_auc: 0.9883 - val_loss: 1.8749 - val_precision: 0.9688 - val_recall: 0.9688 - learning_rate: 9.7235e-05

Epoch 15: LearningRateScheduler setting learning rate to 9.944703981280327e-05.
Epoch 15/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6440 - auc: 0.8506 - loss: 1.9993 - precision: 0.9723 - recall: 0.6159
Epoch 15: val_auc did not improve from 0.98828

Epoch 15: ReduceLROnPlateau reducing learning rate to 6.961292747291735e-05.
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.6452 - auc: 0.8486 - loss: 2.0005 - precision: 0.9729 - recall: 0.6167 - val_accuracy: 0.9444 - val_auc: 0.9766 - val_loss: 1.8713 - val_precision: 0.9688 - val_recall: 0.9688 - learning_rate: 9.9447e-05

Epoch 16: LearningRateScheduler setting learning rate to 0.0001.
Epoch 16/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6510 - auc: 0.8